## Load packages and initialize GEE

In [1]:
# import packages
import ee
import geemap
import pandas as pd

In [2]:
# authenticate the EE api
ee.Authenticate()

True

In [3]:
# initialize the EE api
ee.Initialize(project='ee-bermane')

## Define GEE datasets

In [ ]:
# define EE datasets
# biomass
biomass = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")
rsr = ee.Image("projects/ee-bermane/assets/Root_shoot_ratio_Map_Merged")

# soc
soc_sothe = ee.Image(
    "projects/ee-bermane/assets/McMaster_WWFCanada_soil_carbon1m_250m_kg-m2_version3")
soc_0_10_olm_global = ee.Image(
    "projects/ee-bermane/assets/soc_0_10cm_kg_m2_olm_global")
soc_10_30_olm_global = ee.Image(
    "projects/ee-bermane/assets/soc_10_30cm_kg_m2_olm_global")
soc_30_60_olm_global = ee.Image(
    "projects/ee-bermane/assets/soc_30_60cm_kg_m2_olm_global")
soc_60_100_olm_global = ee.Image(
    "projects/ee-bermane/assets/soc_60_100cm_kg_m2_olm_global")

# peatlands
peat = ee.Image("projects/sat-io/open-datasets/GLOBAL-PEATLAND-DATABASE")

# landcover
lc = ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS")
wte = ee.Image("projects/ee-bermane/assets/WTE_2020")
k3 = ee.Image("projects/ee-bermane/assets/k3_binary").unmask()

# global landcover 2022
lc_cci = (
    ee.ImageCollection("projects/sat-io/open-datasets/ESA/C3S-LC-L4-LCCS")
    .filter(ee.Filter.stringContains("system:index", "2022")).first() #2022
)

# dem mask for land surface
dem_mask = ee.Image('NASA/NASADEM_HGT/001').select('elevation').gt(0)

# netflux
netflux = ee.ImageCollection(
    "projects/wri-datalab/gfw-data-lake/net-flux-forest-extent-per-ha-v1-3-2-2001-2023/net-flux-global-forest-extent-per-ha-2001-2023"
)

# irrecoverable carbon (Noon layers)
irr_carbon_biomass = ee.Image("projects/sat-io/open-datasets/irrecoverable_carbon/carbon_biomass/carbon_biomass_2018")
irr_carbon_soc = ee.Image("projects/sat-io/open-datasets/irrecoverable_carbon/carbon_soil/carbon_soil_2018")
irr_carbon_total = ee.Image("projects/sat-io/open-datasets/irrecoverable_carbon/carbon_total/carbon_total_2018")

# vector
y2y = ee.FeatureCollection("projects/ee-bermane/assets/y2y")
y2y_ecoregions = ee.FeatureCollection(
    "projects/ee-bermane/assets/y2y_ecoregions")
y2y_biomes = ee.FeatureCollection("projects/ee-bermane/assets/y2y_biomes")
rra = ee.FeatureCollection("projects/ee-bermane/assets/ross_river_ipca")
countries = ee.FeatureCollection("USDOS/LSIB/2017")
us_can = ee.FeatureCollection("projects/ee-bermane/assets/us_can_simple")

## Calculate global biomass

In [5]:
# calc ESA CCI biomass
# grab 2022 AGB images
agb = biomass.filter(ee.Filter.stringContains("system:index", "2022")).first()

# create image for litter using Harris ratio (4%)
# leave out dead wood since it is likely sensed already
# using global LC raster now
# mask litter only for forested LC types
#   '006400', // 50: Tree cover, broadleaved, evergreen, closed to open (>15%)
#   '00a000', // 60: Tree cover, broadleaved, deciduous, closed to open (>15%)
#   '003c00', // 70: Tree cover, needleleaved, evergreen, closed to open (>15%)
#   '285000', // 80: Tree cover, needleleaved, deciduous, closed to open (>15%)
#   '788200', // 90: Tree cover, mixed leaf type
#   '8ca000', // 100: Mosaic tree and shrub (>50%) / herbaceous cover (<50%)
#   '00785a', // 160: Tree cover, flooded, fresh or brackish water
#   '009678', // 170: Tree cover, flooded, saline water
#   '0046c8', // 210: Water bodies
#   'ffffff'  // 220: Permanent snow and ice

litter_mask = (
    lc_cci.eq(50)
    .Or(lc_cci.eq(60))
    .Or(lc_cci.eq(70))
    .Or(lc_cci.eq(80))
    .Or(lc_cci.eq(90))
    .Or(lc_cci.eq(100))
    .Or(lc_cci.eq(160))
    .Or(lc_cci.eq(170))
)

litter = agb.select(['AGB']).multiply(0.04).multiply(litter_mask)

# create image for BGB (using global rsr map)
bgb = agb.select(['AGB']).multiply(rsr).rename('BGB')

# add AGB + litter and BGB together
bio = agb.select(['AGB']).add(litter).addBands(
    bgb).rename(['agb_t_ha', 'bgb_t_ha'])

# multiply values by 0.47 to get carbon density
# 0.47 used by Harris et al. (2021)
# mask water (210) and snow/ice (220)
bio = bio.multiply(0.47).updateMask(
    lc_cci.neq(210).And(lc_cci.neq(220)))

# compute per-pixel area in ha
pixel_area_ha = ee.Image.pixelArea().divide(10000)

# Create carbon layer mask to filter pixel area raster
carbon_mask = bio.select(['agb_t_ha']).mask().neq(0)

# Mask pixel_area_ha to carbon layers
pixel_area_agb_extent = pixel_area_ha.updateMask(carbon_mask)

# calculate total biomass c per pixel and rename bands
bio_stock = bio.multiply(
    pixel_area_ha
).rename(
    ['agb_t', 'bgb_t']
).addBands(
    pixel_area_agb_extent.rename(['pixel_area_agb_extent_ha'])
).addBands(
    pixel_area_ha.rename(['pixel_area_ha'])
)

In [6]:
# # export bio raster to drive
# ee.batch.Export.image.toDrive(
#     image=bio_2021.select(['agb_t_ha']),
#     description='bio',
#     region=y2y.geometry(),
#     scale=30,
#     maxPixels=1e13,
#     fileNamePrefix='bio'
# ).start()


## Calculate soil carbon from Sothe Canada data and Open Land Map Global data

In [7]:
# calc open land map soc
# add the layers together from 0-100cm
soc_olm = (
    soc_0_10_olm_global
    .add(soc_10_30_olm_global)
    .add(soc_30_60_olm_global)
    .add(soc_60_100_olm_global)
    )

# multiply by 10 to get t/ha
# mask water/snow/ice
soc_olm = soc_olm.multiply(10).rename(
    'soc_dens').updateMask(lc_cci.neq(210).And(lc_cci.neq(220)))

# reproject to match sothe to blend images
soc_olm_reproj_sothe = soc_olm.resample('bilinear').toFloat()

In [8]:
# calc sothe soc
# multiply by 10 to get t/ha)
# mask water/snow/ice
soc_sothe = soc_sothe.multiply(10).rename(
    'soc_dens').updateMask(lc_cci.neq(210).And(lc_cci.neq(220)))

# blend sothe and olm carbon across y2y
soc_blend = ee.ImageCollection([soc_olm_reproj_sothe, soc_sothe]).mosaic().rename('soc_t_ha').reproject(
    crs=soc_sothe.projection(),
    crsTransform=soc_sothe.projection().getInfo().get('transform')
).updateMask(lc_cci.neq(210).And(lc_cci.neq(220)))  # mask snow/ice, water

# Create soc layer mask to filter pixel area raster
soc_mask = soc_blend.mask().neq(0)

# Mask pixel_area_ha to carbon layers
pixel_area_soc_extent = pixel_area_ha.updateMask(soc_mask)

# multiply by pixel area to get total carbon per pixel
soc_blend_stock = soc_blend.multiply(pixel_area_ha).rename('soc_t').addBands(
    pixel_area_soc_extent.rename(['pixel_area_soc_extent_ha'])
).addBands(
    pixel_area_ha.rename(['pixel_area_ha'])
)

## Calculate average carbon density in mountains vs. non-mountains globally

Density calculations now involve all landmass

In [9]:
# create a global geometry to reduce over
globe = ee.Geometry.Rectangle(
    coords=[-180, -90, 180, 90],
    proj=ee.Projection('EPSG:4326'),  # Explicitly defining the projection,
    geodesic=False)

In [10]:
# do the same for total carbon stock
biomass_k3 = (
    bio
    .reduce(ee.Reducer.sum()) # sum biomass layers
    .multiply(pixel_area_ha)
    .updateMask(k3) # mask for mountains only
    .rename(['bio_t'])
    .reduceRegion( # calc sum
        reducer=ee.Reducer.sum(),
        geometry=globe,
        scale=bio.projection().nominalScale(),
        maxPixels=1e20
    )
)

biomass_not_k3 = (
    bio
    .reduce(ee.Reducer.sum()) # sum biomass layers
    .multiply(pixel_area_ha)
    .updateMask(k3.Not()) # mask for not mountains
    .rename(['bio_t'])
    .reduceRegion( # calc sum
        reducer=ee.Reducer.sum(),
        geometry=globe,
        scale=bio.projection().nominalScale(),
        maxPixels=1e20
    )
)

soc_k3 = (
    soc_blend
    .multiply(pixel_area_ha)
    .updateMask(k3) # mask for mountains only
    .reduceRegion( # calc sum
        reducer=ee.Reducer.sum(),
        geometry=globe,
        scale=soc_blend.projection().nominalScale(),
        maxPixels=1e20
    )
)

soc_not_k3 = (
    soc_blend
    .multiply(pixel_area_ha)
    .updateMask(k3.Not()) # mask for not mountains
    .reduceRegion( # calc sum
        reducer=ee.Reducer.sum(),
        geometry=globe,
        scale=soc_blend.projection().nominalScale(),
        maxPixels=1e20
    )
)

# export to drive
ee.batch.Export.table.toDrive(
    collection=ee.FeatureCollection([
        ee.Feature(None, {
            "biomass_k3": biomass_k3,
            "biomass_not_k3": biomass_not_k3,
            "soc_k3": soc_k3, 
            "soc_not_k3": soc_not_k3
            })]),
    description="carbon_stock_k3_stats_global",
    folder="",
    fileFormat="CSV"
).start()

## Calculate irrecoverable carbon in mountains (Noon Layers)

In [11]:
# carbon density 2018
# mask irr_carbon_biomass with k3 mountain layer
irr_carbon_biomass_k3 = (
    irr_carbon_biomass
    .updateMask(k3) # mask for mountains only
    .reduceRegion( # calc mean density
        reducer=ee.Reducer.mean(),
        geometry=globe,
        scale=irr_carbon_biomass.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_biomass with not k3 mountain layer
irr_carbon_biomass_not_k3 = (
    irr_carbon_biomass
    .updateMask(k3.Not()) # mask for not mountains only
    .reduceRegion( # calc mean density
        reducer=ee.Reducer.mean(),
        geometry=globe,
        scale=irr_carbon_biomass.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_biomass with y2y
irr_carbon_biomass_y2y = (
    irr_carbon_biomass
    .reduceRegion( # calc mean density
        reducer=ee.Reducer.mean(),
        geometry=y2y.geometry(),
        scale=irr_carbon_biomass.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_soc with k3 mountain layer
irr_carbon_soc_k3 = (
    irr_carbon_soc
    .updateMask(k3) # mask for mountains only
    .reduceRegion( # calc mean density
        reducer=ee.Reducer.mean(),
        geometry=globe,
        scale=irr_carbon_soc.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_soc with not k3 mountain layer
irr_carbon_soc_not_k3 = (
    irr_carbon_soc
    .updateMask(k3.Not()) # mask for not mountains only
    .reduceRegion( # calc mean density
        reducer=ee.Reducer.mean(),
        geometry=globe,
        scale=irr_carbon_soc.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_soc with y2y
irr_carbon_soc_y2y = (
    irr_carbon_soc
    .reduceRegion( # calc mean density
        reducer=ee.Reducer.mean(),
        geometry=y2y.geometry(),
        scale=irr_carbon_soc.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_total with k3 mountain layer
irr_carbon_total_k3 = (
    irr_carbon_total
    .updateMask(k3) # mask for mountains only
    .reduceRegion( # calc mean density
        reducer=ee.Reducer.mean(),
        geometry=globe,
        scale=irr_carbon_total.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_total with not k3 mountain layer
irr_carbon_total_not_k3 = (
    irr_carbon_total
    .updateMask(k3.Not()) # mask for not mountains only
    .reduceRegion( # calc mean density
        reducer=ee.Reducer.mean(),
        geometry=globe,
        scale=irr_carbon_total.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_total with y2y
irr_carbon_total_y2y = (
    irr_carbon_total
    .reduceRegion( # calc mean density
        reducer=ee.Reducer.mean(),
        geometry=y2y.geometry(),
        scale=irr_carbon_total.projection().nominalScale(),
        maxPixels=1e20
    )
)

# export to drive
ee.batch.Export.table.toDrive(
    collection=ee.FeatureCollection([
        ee.Feature(None, {
            "irr_carbon_biomass_k3": irr_carbon_biomass_k3,
            "irr_carbon_biomass_not_k3": irr_carbon_biomass_not_k3,
            "irr_carbon_biomass_y2y": irr_carbon_biomass_y2y,
            "irr_carbon_soc_k3": irr_carbon_soc_k3,
            "irr_carbon_soc_not_k3": irr_carbon_soc_not_k3,
            "irr_carbon_soc_y2y": irr_carbon_soc_y2y,
            "irr_carbon_total_k3": irr_carbon_total_k3,
            "irr_carbon_total_not_k3": irr_carbon_total_not_k3,
            "irr_carbon_total_y2y": irr_carbon_total_y2y
            })]),
    description="noon_global_irr_carbon_density_k3_stats",
    folder="",
    fileFormat="CSV"
).start()

In [12]:
# do the same for total carbon
# can use the same pixel_area_ha since irrecoverable carbon
# layers are also in ha

# mask irr_carbon_biomass with k3 mountain layer
irr_carbon_biomass_k3 = (
    irr_carbon_biomass.multiply(pixel_area_ha)
    .updateMask(k3) # mask for mountains only
    .reduceRegion( # calc total stock
        reducer=ee.Reducer.sum(),
        geometry=globe,
        scale=irr_carbon_biomass.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_biomass with not k3 mountain layer
irr_carbon_biomass_not_k3 = (
    irr_carbon_biomass.multiply(pixel_area_ha)
    .updateMask(k3.Not()) # mask for not mountains only
    .reduceRegion( # calc total stock
        reducer=ee.Reducer.sum(),
        geometry=globe,
        scale=irr_carbon_biomass.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_biomass with y2y
irr_carbon_biomass_y2y = (
    irr_carbon_biomass.multiply(pixel_area_ha)
    .reduceRegion( # calc total stock
        reducer=ee.Reducer.sum(),
        geometry=y2y.geometry(),
        scale=irr_carbon_biomass.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_soc with k3 mountain layer
irr_carbon_soc_k3 = (
    irr_carbon_soc.multiply(pixel_area_ha)
    .updateMask(k3) # mask for mountains only
    .reduceRegion( # calc total stock
        reducer=ee.Reducer.sum(),
        geometry=globe,
        scale=irr_carbon_soc.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_soc with not k3 mountain layer
irr_carbon_soc_not_k3 = (
    irr_carbon_soc.multiply(pixel_area_ha)
    .updateMask(k3.Not()) # mask for not mountains only
    .reduceRegion( # calc total stock
        reducer=ee.Reducer.sum(),
        geometry=globe,
        scale=irr_carbon_soc.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_soc with y2y
irr_carbon_soc_y2y = (
    irr_carbon_soc.multiply(pixel_area_ha)
    .reduceRegion( # calc total stock
        reducer=ee.Reducer.sum(),
        geometry=y2y.geometry(),
        scale=irr_carbon_soc.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_total with k3 mountain layer
irr_carbon_total_k3 = (
    irr_carbon_total.multiply(pixel_area_ha)
    .updateMask(k3) # mask for mountains only
    .reduceRegion( # calc total stock
        reducer=ee.Reducer.sum(),
        geometry=globe,
        scale=irr_carbon_total.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_total with not k3 mountain layer
irr_carbon_total_not_k3 = (
    irr_carbon_total.multiply(pixel_area_ha)
    .updateMask(k3.Not()) # mask for not mountains only
    .reduceRegion( # calc total stock
        reducer=ee.Reducer.sum(),
        geometry=globe,
        scale=irr_carbon_total.projection().nominalScale(),
        maxPixels=1e20
    )
)

# mask irr_carbon_total with y2y
irr_carbon_total_y2y = (
    irr_carbon_total.multiply(pixel_area_ha)
    .reduceRegion( # calc total stock
        reducer=ee.Reducer.sum(),
        geometry=y2y.geometry(),
        scale=irr_carbon_total.projection().nominalScale(),
        maxPixels=1e20
    )
)

# export to drive
ee.batch.Export.table.toDrive(
    collection=ee.FeatureCollection([
        ee.Feature(None, {
            "irr_carbon_biomass_k3": irr_carbon_biomass_k3,
            "irr_carbon_biomass_not_k3": irr_carbon_biomass_not_k3,
            "irr_carbon_biomass_y2y": irr_carbon_biomass_y2y,
            "irr_carbon_soc_k3": irr_carbon_soc_k3,
            "irr_carbon_soc_not_k3": irr_carbon_soc_not_k3,
            "irr_carbon_soc_y2y": irr_carbon_soc_y2y,
            "irr_carbon_total_k3": irr_carbon_total_k3,
            "irr_carbon_total_not_k3": irr_carbon_total_not_k3,
            "irr_carbon_total_y2y": irr_carbon_total_y2y
            })]),
    description="noon_global_irr_carbon_stock_k3_stats",
    folder="",
    fileFormat="CSV"
).start()

## General Area Stats

In [13]:
# Y2Y in k3
y2y_k3 = (
    pixel_area_ha
    .updateMask(k3) # mask for mountains only
    .reduceRegion( # calc total ha
        reducer=ee.Reducer.sum(),
        geometry=y2y.geometry(),
        scale=100,
        maxPixels=1e20
    )
)

# Y2Y not in k3
y2y_not_k3 = (
    pixel_area_ha
    .updateMask(k3.Not()) # mask for not mountains only
    .reduceRegion( # calc total ha
        reducer=ee.Reducer.sum(),
        geometry=y2y.geometry(),
        scale=100,
        maxPixels=1e20
    )
)

print(y2y_k3.getInfo())
print(y2y_not_k3.getInfo())

{'area': 121165313.08041003}
{'area': 15829621.282585807}


In [14]:
# Global in k3
global_k3 = (
    pixel_area_ha
    .updateMask(k3) # mask for mountains only
    .updateMask(dem_mask) # mask for global terrestrial land surface (not water)
    .reduceRegion( # calc total ha
        reducer=ee.Reducer.sum(),
        geometry=globe,
        scale=100,
        maxPixels=1e20
    )
)

# global not in k3
global_not_k3 = (
    pixel_area_ha
    .updateMask(k3.Not()) # mask for not mountains only
    .updateMask(dem_mask) # mask for global terrestrial land surface (not water)
    .reduceRegion( # calc total ha
        reducer=ee.Reducer.sum(),
        geometry=globe,
        scale=100,
        maxPixels=1e20
    )
)

# export to drive
ee.batch.Export.table.toDrive(
    collection=ee.FeatureCollection([
        ee.Feature(None, {
            "area_ha_k3": global_k3,
            "area_ha_not_k3": global_not_k3
            })]),
    description="global_k3_area_stats",
    folder="",
    fileFormat="CSV"
).start()

## Add layers to map

In [23]:
# set biomass palette
biomass_palette = [
    "#C6ECAE", "#A1D490", "#7CB970", "#57A751", "#348E32",
    "#267A29", "#176520", "#0C4E15", "#07320D", "#031807"
]

# set land cover palette
lc_palette = [
    "#033e00",  # Temperate or sub-polar needleleaf forest
    "#939b71",  # Sub-polar taiga needleleaf forest
    "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
    "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
    "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
    "#6b7d2c",  # Mixed forest
    "#b29d29",  # Tropical or sub-tropical shrubland
    "#b48833",  # Temperate or sub-polar shrubland
    "#e9da5d",  # Tropical or sub-tropical grassland
    "#e0cd88",  # Temperate or sub-polar grassland
    "#a07451",  # Sub-polar or polar shrubland-lichen-moss
    "#bad292",  # Sub-polar or polar grassland-lichen-moss
    "#3f8970",  # Sub-polar or polar barren-lichen-moss
    "#6ca289",  # Wetland
    "#e6ad6a",  # Cropland
    "#a9abae",  # Barren land
    "#db2126",  # Urban and built-up
    "#4c73a1",  # Water
    "#fff7fe",  # Snow and ice
]

In [28]:
# create a map
m = geemap.Map()

# add layers
# m.addLayer(soc_blend, {"min": 1, "max": 450, "palette": biomass_palette}, "SOC Density Blend 0-1m")
# m.addLayer(soc_olm, {"min": 1, "max": 450, "palette": biomass_palette}, "SOC OLM Density 0-1m")
# m.addLayer(soc_olm_reproj_sothe, {"min": 1, "max": 450, "palette": biomass_palette}, "SOC OLM Density 0-1m")
# m.addLayer(soc_sothe, {"min": 1, "max": 450, "palette": biomass_palette}, "SOC Sothe Density 0-1m")
# m.addLayer(rsr, {}, 'Root-to-Shoot Ratio Global')
# m.addLayer(bio_2021.select(['agb_t_ha']).clip(y2y), {
#           "min": 1, "max": 100, "palette": biomass_palette}, 'CCI AGB Density')
# m.addLayer(bio_2021.select(['dpm_t_ha']).clip(y2y), {"min": 1, "max": 60, "palette": biomass_palette}, 'DPM Density')
# m.addLayer(pixel_area_agb_extent.clip(y2y), {}, 'Carbon Mask')
# m.addLayer(lc, {"min": 1, "max": 19,
#                 "palette": lc_palette}, 'Landcover')
# m.addLayer(lc.mask(), {}, 'LC Mask')
# m.addLayer(y2y_can, {}, 'Y2Y Canada Extent')
# m.addLayer(big_fc, {}, 'Big Geo')
# m.addLayer(bio.select(['agb_t_ha']), {
#           "min": 1, "max": 100, "palette": biomass_palette}, 'CCI AGB Density')
# m.addLayer(agb_2021.select(['AGB']), {
#     "min": 1, "max": 100, "palette": biomass_palette}, 'CCI AGB Density')
m.addLayer(k3.unmask(), {"min": 0, "max": 1}, 'k3 Mountain Classification')
m.addLayer(k3.unmask().Not(), {"min": 0, "max": 1}, 'Not k3 Mountain Classification')
# m.addLayer(wte.updateMask(lc.mask()), {"min": 1, "max": 450, "palette": biomass_palette}, "WTE")
# m.addLayer(irr_carbon_biomass, {"min": 1, "max": 450, "palette": biomass_palette}, "Irrecoverable Carbon Biomass 2018")
# m.addLayer(irr_carbon_soc, {"min": 1, "max": 450, "palette": biomass_palette}, "Irrecoverable Carbon SOC 2018")
# m.addLayer(irr_carbon.updateMask(k3), {"min": 1, "max": 450, "palette": biomass_palette}, "Irrecoverable Carbon k3 2018")
# m.addLayer(irr_carbon.updateMask(k3.Not()), {"min": 1, "max": 450, "palette": biomass_palette}, "Irrecoverable Carbon Not k3 2018")
# m.addLayer(countries, {}, 'Countries')
# m.addLayer(us_can, {}, 'US and Canada') 
m.addLayer(pixel_area_ha.updateMask(k3.Not()), {}, "Pixel Area Ha in k3")

# Display the map
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…